In [1]:
"""%load_ext autoreload
%autoreload 2
%load_ext wurlitzer
"""
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import xobjects as xo
import xpart as xp
import xtrack as xt
import xcoll as xc

In [2]:
context = xo.ContextCpu()
# context = xo.ContextCupy()
# context = xo.ContextPyopencl()

# On a modern CPU, we get ~5000 particle*turns/s
# So this script should take around half an hour
beam          =  1
plane         = 'V'

num_turns     = 20
num_particles = 20000
engine        = 'everest'

path_in  = Path("/eos/project-c/collimation-team/machine_configurations/LHC_run3/2024")

# Load from json
line = xt.Line.from_json(path_in / 'xsuite' / f'levelling.20_b{beam}.json')

Loading line from dict:   0%|          | 0/175585 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
# Initialise collmanager
# coll_manager = xc.CollimatorManager.from_yaml(path_in / 'colldbs' / f'levelling.20.yaml', line=line,
#                                               beam=beam, ignore_crystals=False, record_impacts=False)
coll_manager = xc.CollimatorManager.from_yaml(xc._pkg_root / '..' / 'examples' / 'colldb' / 'lhc_run3_crystals.yaml', line=line,
                                              beam=beam, ignore_crystals=False, record_impacts=['tcp.c6l7.b1', 'tcp.d6l7.b1'])

# Install collimators into line
if engine == 'everest':
    coll_manager.install_everest_collimators(verbose=True)
elif engine == 'black':
    coll_manager.install_black_absorbers(verbose=True)
else:
    raise ValueError(f"Unknown scattering engine {engine}!")

# Aperture model check
print('\nAperture model check after introducing collimators:')
df_with_coll = line.check_aperture()
assert not np.any(df_with_coll.has_aperture_problem)

Compiling ContextCpu kernels...


ff2520c09af347f891f8355cd1e83c5d.c: In function 'scatter_cry':
ff2520c09af347f891f8355cd1e83c5d.c:9926:28: warning: unused variable 'x_P' [-Wunused-variable]
 9926 |                     double x_P = -s_P_tmp*sin(tilt_int) + x_P_tmp*cos(tilt_int);
      |                            ^~~
ff2520c09af347f891f8355cd1e83c5d.c:9925:28: warning: unused variable 's_P' [-Wunused-variable]
 9925 |                     double s_P = s_P_tmp*cos(tilt_int) + x_P_tmp*sin(tilt_int);
      |                            ^~~
ff2520c09af347f891f8355cd1e83c5d.c:9823:18: warning: unused variable 'cry_spTilt' [-Wunused-variable]
 9823 |     double const cry_spTilt = sin(cry_tilt);
      |                  ^~~~~~~~~~
ff2520c09af347f891f8355cd1e83c5d.c:9813:12: warning: unused variable 'offset' [-Wunused-variable]
 9813 |     double offset   = everest->coll->offset;
      |            ^~~~~~


Done compiling ContextCpu kernels.
Installing tcpcv.a6l7.b1        as EverestCrystal
Installing tcpch.a4l7.b1        as EverestCrystal
Compiling ContextCpu kernels...


94b252230d72476e9194812718c9d65e.c: In function 'scatter_cry':
94b252230d72476e9194812718c9d65e.c:9926:28: warning: unused variable 'x_P' [-Wunused-variable]
 9926 |                     double x_P = -s_P_tmp*sin(tilt_int) + x_P_tmp*cos(tilt_int);
      |                            ^~~
94b252230d72476e9194812718c9d65e.c:9925:28: warning: unused variable 's_P' [-Wunused-variable]
 9925 |                     double s_P = s_P_tmp*cos(tilt_int) + x_P_tmp*sin(tilt_int);
      |                            ^~~
94b252230d72476e9194812718c9d65e.c:9823:18: warning: unused variable 'cry_spTilt' [-Wunused-variable]
 9823 |     double const cry_spTilt = sin(cry_tilt);
      |                  ^~~~~~~~~~
94b252230d72476e9194812718c9d65e.c:9813:12: warning: unused variable 'offset' [-Wunused-variable]
 9813 |     double offset   = everest->coll->offset;
      |            ^~~~~~


Done compiling ContextCpu kernels.
Installing tcl.4r1.b1           as EverestCollimator
Installing tcl.5r1.b1           as EverestCollimator
Installing tcl.6r1.b1           as EverestCollimator
Installing tctph.4l2.b1         as EverestCollimator
Installing tctpv.4l2.b1         as EverestCollimator
Installing tdisa.a4l2.b1        as EverestCollimator
Installing tdisb.a4l2.b1        as EverestCollimator
Installing tdisc.a4l2.b1        as EverestCollimator
Installing tclia.4r2            as EverestCollimator
Installing tclib.6r2.b1         as EverestCollimator
Installing tcld.a11r2.b1        as EverestCollimator
Installing tcp.6l3.b1           as EverestCollimator
Installing tcsg.5l3.b1          as EverestCollimator
Installing tcsg.4r3.b1          as EverestCollimator
Installing tcsg.a5r3.b1         as EverestCollimator
Installing tcsg.b5r3.b1         as EverestCollimator
Installing tcla.a5r3.b1         as EverestCollimator
Installing tcla.b5r3.b1         as EverestCollimator
Installing 

Checking aperture:   0%|          | 0/175136 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]


In [4]:
line.build_tracker()

Compiling ContextCpu kernels...


8b310e6f9c5d4a0fb9e2f51de38be151.c: In function 'scatter_cry':
8b310e6f9c5d4a0fb9e2f51de38be151.c:14594:28: warning: unused variable 'x_P' [-Wunused-variable]
14594 |                     double x_P = -s_P_tmp*sin(tilt_int) + x_P_tmp*cos(tilt_int);
      |                            ^~~
8b310e6f9c5d4a0fb9e2f51de38be151.c:14593:28: warning: unused variable 's_P' [-Wunused-variable]
14593 |                     double s_P = s_P_tmp*cos(tilt_int) + x_P_tmp*sin(tilt_int);
      |                            ^~~
8b310e6f9c5d4a0fb9e2f51de38be151.c:14491:18: warning: unused variable 'cry_spTilt' [-Wunused-variable]
14491 |     double const cry_spTilt = sin(cry_tilt);
      |                  ^~~~~~~~~~
8b310e6f9c5d4a0fb9e2f51de38be151.c:14481:12: warning: unused variable 'offset' [-Wunused-variable]
14481 |     double offset   = everest->coll->offset;
      |            ^~~~~~


Done compiling ContextCpu kernels.


In [5]:
coll_manager.set_openings({'tcp.d6l7.b1': 5, 'tcp.c6l7.b1': 5, 'tcpcv.a6l7.b1': 5.5})

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [6]:
line['tcpch.a4l7.b1'].active = True
line['tcpcv.a6l7.b1'].active = True
line['tcp.b6l7.b1'].record_touches = True
line['tcp.c6l7.b1'].record_touches = True
line['tcp.d6l7.b1'].record_touches = True
line['tdisa.a4l2.b1'].record_touches = True

In [7]:
line['tcpch.a4l7.b1'].to_dict()

{'__class__': 'EverestCrystal',
 'length': 0.004,
 'active': 1,
 'record_touches': 0,
 'record_interactions': 0,
 'align_angle': 1.2112695362072954e-05,
 '_critical_angle': 0.0,
 'xdim': 0.002,
 'ydim': 0.05,
 'thick': 0.0,
 'miscut': 0.0,
 'rutherford_rng': {'__class__': 'RandomRutherford',
  'lower_val': 0.0009982,
  'upper_val': 0.02,
  'A': 0.0016160247264725453,
  'B': 166.49518410000002,
  'Newton_iterations': 7},
 '_tracking': 0,
 'angle': 0.0,
 'jaw': [0.0020206608911066404, 0.02499999999999991],
 'tilt': 0.0,
 'side': 'left',
 'lattice': 'strip',
 'material': {'__class__': 'CrystalMaterial',
  'Z': 14.0,
  'A': 28.08,
  'density': 2.33,
  'excitation_energy': 1.73e-07,
  'nuclear_radius': 0.441,
  'nuclear_elastic_slope': 120.14,
  'cross_section': array([6.64e-01, 4.30e-01, 0.00e+00, 0.00e+00, 0.00e+00, 3.90e-04]),
  'hcut': 0.02,
  'name': 'Silicon',
  '_only_mcs': 0,
  'crystal_radiation_length': 0.0937,
  'crystal_nuclear_length': 0.4652,
  'crystal_plane_distance': 9.6e-0

In [8]:
tw = line.twiss()

In [9]:
coll = 'tcpch.a4l7.b1'
ang = line[coll].angle
ref = np.cos(ang)* tw.rows[coll]['x'] + np.sin(ang)* tw.rows[coll]['y']
sigx = np.sqrt(3.5E-6*tw.rows[coll]['betx']/7247.36468857)
sigy = np.sqrt(3.5E-6*tw.rows[coll]['bety']/7247.36468857)
sig = np.sqrt(np.cos(ang)**2 * sigx**2 + np.sin(ang)**2 * sigy**2)
print(ref + sig*5)

[0.00202066]


In [10]:
part = xc.generate_pencil_on_collimator(line, 'tcp.d6l7.b1', 100, nemitt_x=3.5e-6, nemitt_y=3.5e-6)

Collimator tcp.d6l7.b1 is diverging.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [11]:
coll_manager._io_buffer

<BufferNumpy 999992/241000752>

In [12]:
part._xobject._io_buffer

AttributeError: 'ParticlesData' object has no attribute '_io_buffer'

In [13]:
coll_manager.impacts._index.buffer_id

0

In [14]:
# # Optimise the line
# line.optimize_for_tracking()
# idx = line.element_names.index(tcp)
# part.at_element = idx
# part.start_tracking_at_element = idx

# line.discard_tracker()
# line.build_tracker(_context=xo.ContextCpu(omp_num_threads=28))

# Track
coll_manager.enable_scattering()
line.track(part, num_turns=20, time=True, with_progress=1)
coll_manager.disable_scattering()
print(f"Done tracking in {line.time_last_track:.1f}s.")

# line.discard_tracker()
# line.build_tracker(_context=xo.ContextCpu())

Tracking:   0%|          | 0/20 [00:00<?, ?it/s]

Error: buffer_id mismatch!
Error: buffer_id mismatch!
Error: buffer_id mismatch!
Error: buffer_id mismatch!
Done tracking in 0.0s.


In [15]:
# Save lossmap to json, which can be loaded, combined (for more statistics),
# and plotted with the 'lossmaps' package
line_is_reversed = True if f'{beam}' == '2' else False
ThisLM = xc.LossMap(line, line_is_reversed=line_is_reversed, part=part)
path_out = Path.cwd()
ThisLM.save_summary(file=Path(path_out, f'coll_summary_B{beam}{plane}.out'))
# Save a summary of the collimator losses to a text file
print(ThisLM.summary)

         collname  nabs  length             s               type
0      tcl.4r1.b1   0.0   1.000    150.030000  EverestCollimator
1      tcl.5r1.b1   0.0   1.000    184.357000  EverestCollimator
2      tcl.6r1.b1   0.0   1.000    219.013000  EverestCollimator
3    tctph.4l2.b1   0.0   1.000   3213.903584  EverestCollimator
4    tctpv.4l2.b1   0.0   1.000   3215.903583  EverestCollimator
5   tdisa.a4l2.b1  89.0   1.565   3249.693583  EverestCollimator
6   tdisb.a4l2.b1   0.0   1.565   3251.273583  EverestCollimator
7   tdisc.a4l2.b1   0.0   1.565   3252.853583  EverestCollimator
8       tclia.4r2   0.0   1.000   3403.984583  EverestCollimator
9    tclib.6r2.b1   0.0   1.000   3560.090583  EverestCollimator
10  tcld.a11r2.b1   0.0   0.600   3758.149975  EverestCollimator
11     tcp.6l3.b1   0.0   0.600   6487.671299  EverestCollimator
12    tcsg.5l3.b1   0.0   1.000   6520.992797  EverestCollimator
13    tcsg.4r3.b1   0.0   1.000   6707.575797  EverestCollimator
14   tcsg.a5r3.b1   0.0  

In [13]:
coll_manager.impacts.to_pandas()

,turn,collimator,interaction_type,ds,parent_id,parent_x,parent_px,parent_y,parent_py,parent_zeta,...,child_y,child_py,child_zeta,child_delta,child_energy,child_mass,child_charge,child_z,child_a,child_pdgid


In [15]:
coll_manager.impacts.to_pandas().columns

Index(['turn', 'collimator', 'interaction_type', 'ds', 'parent_id', 'parent_x',
       'parent_px', 'parent_y', 'parent_py', 'parent_zeta', 'parent_delta',
       'parent_energy', 'parent_mass', 'parent_charge', 'parent_z', 'parent_a',
       'parent_pdgid', 'child_id', 'child_x', 'child_px', 'child_y',
       'child_py', 'child_zeta', 'child_delta', 'child_energy', 'child_mass',
       'child_charge', 'child_z', 'child_a', 'child_pdgid'],
      dtype='object')

In [14]:
coll_manager.lossmap(part, file=None)

{'collimator': {'s': [219.013,
   6520.992797083998,
   19789.18438252112,
   19791.184382375122,
   19793.184382229123,
   19832.67888061313,
   19891.906380613127,
   19895.906380613127,
   19917.23638061313,
   19989.16238061313,
   19991.16238061313,
   19995.16238061313,
   20086.41838061313,
   20102.41838061313,
   20108.41838061313,
   20148.089380613128,
   20178.96338010613,
   20212.23237811213,
   20231.86037811213,
   26510.938177061144,
   26512.938176726144],
  'name': ['tcl.6r1.b1',
   'tcsg.5l3.b1',
   'tcp.d6l7.b1',
   'tcp.c6l7.b1',
   'tcp.b6l7.b1',
   'tcsg.a6l7.b1',
   'tcsg.b5l7.b1',
   'tcsg.a5l7.b1',
   'tcsg.d4l7.b1',
   'tcspm.b4l7.b1',
   'tcsg.a4l7.b1',
   'tcsg.a4r7.b1',
   'tcsg.b5r7.b1',
   'tcsg.d5r7.b1',
   'tcspm.e5r7.b1',
   'tcla.a6r7.b1',
   'tcla.b6r7.b1',
   'tcla.c6r7.b1',
   'tcla.a7r7.b1',
   'tctph.4l1.b1',
   'tctpv.4l1.b1'],
  'length': [1.0,
   1.0,
   0.6,
   0.6,
   0.6,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   

In [12]:
pdb on

Automatic pdb calling has been turned ON


In [9]:
coll_manager.lossmap(part)

{'collimator': {'s': [19789.18438252112,
   19791.184382375122,
   19793.184382229123,
   19832.67888061313,
   19891.906380613127,
   19895.906380613127,
   19917.23638061313,
   19989.16238061313,
   19991.16238061313,
   19995.16238061313,
   20086.41838061313,
   20102.41838061313,
   20108.41838061313,
   20143.02338061313,
   20148.089380613128,
   20212.23237811213,
   26512.938176726144],
  'name': ['tcp.d6l7.b1',
   'tcp.c6l7.b1',
   'tcp.b6l7.b1',
   'tcsg.a6l7.b1',
   'tcsg.b5l7.b1',
   'tcsg.a5l7.b1',
   'tcsg.d4l7.b1',
   'tcspm.b4l7.b1',
   'tcsg.a4l7.b1',
   'tcsg.a4r7.b1',
   'tcsg.b5r7.b1',
   'tcsg.d5r7.b1',
   'tcspm.e5r7.b1',
   'tcspm.6r7.b1',
   'tcla.a6r7.b1',
   'tcla.c6r7.b1',
   'tctpv.4l1.b1'],
  'length': [0.6,
   0.6,
   0.6,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0],
  'n': [3873.0,
   4.0,
   72.0,
   208.0,
   246.0,
   144.0,
   46.0,
   11.0,
   51.0,
   37.0,
   21.0,
   14.0,
   4.